# Assignment Set 1

### import libraries

In [ ]:
import numpy as np

%matplotlib inline

## problem 1.1: Vibrating String

In [ ]:
from src.set_1.vibrating_string import VibratingString

for sin(5pix)

In [ ]:
def psi_sin_2pi(x):
    return np.sin(2 * np.pi * x)

string = VibratingString(psi_sin_2pi, N=100, simulation_time=1, fig_name="sin_2pi")
string.run_time_stepping()
string.plot_static_simulation()
string.plot_dynamic_simulation()

for sin(5pix)

In [ ]:
def psi_sin_5pi(x):
    return np.sin(5 * np.pi * x)

string = VibratingString(psi_sin_5pi, N=100, simulation_time=1, fig_name="sin_5pi")
string.run_time_stepping()
string.plot_static_simulation()
string.plot_dynamic_simulation()

for sin(5pix) localized between x = 1/5 and x = 2/5

In [ ]:
def psi_localized(x):
    return np.where((x > 1/5) & (x < 2/5), np.sin(5 * np.pi * x), 0)

string = VibratingString(psi_localized, N=100, simulation_time=1, fig_name="localized")
string.run_time_stepping()
string.plot_static_simulation()
string.plot_dynamic_simulation()

## Problem 1.2: The Time Dependent Diffusion Equation 

In [ ]:
from src.set_1.time_dependent_diffusion import TimeDependentDiffusion
from src.set_1.time_independent_diffusion import TimeIndependentDifussion

In [ ]:
simulation = TimeDependentDiffusion(N=50, simulation_time=1, fig_name="diffusion_simulation") 
simulation.run_time_stepping()

In [ ]:
simulation.create_subplot_log_time()

In [ ]:
simulation.check_analytical_solution()

In [ ]:
simulation.create_animation()

In [ ]:
solver = TimeIndependentDifussion(N=50)
solver.solve()
solver.plot_solution()